### Mean Distance + Number of Tiles Calculation Cell

In [ ]:
import numpy as np
import pandas as pd
import ligo.skymap.io
import healpy as hp
import matplotlib.pyplot as plt

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord
import pandas as pd
import numpy as np

import ligo.skymap.io
import healpy as hp
import matplotlib.pyplot as plt



print('Yes')

def get_probabilities(skymap, ra, dec, radius=0.35*u.deg):
    """
    Compute the probabilities covered in a grid of ra, dec with radius
    given a healpix skymap

    Pass the legal RA and Dec lists to this function

    Note: this radius is such that for the default sky grid (nside=128),
    fields overlap such that the area of the sky becomes 1.83 * 4pi steradians.
    As a result, the sum of probabilities will be 1.83
    """
    # fact : int, optional
    # Only used when inclusive=True. The overlapping test will be done at
    # the resolution fact*nside. For NESTED ordering, fact must be a power of 2, less than 2**30,
    # else it can be any positive integer. Default: 4.
    fact = 1
    nside_skymap = hp.npix2nside(len(skymap))
    tile_area = 1#np.pi * radius.to(u.deg).value ** 2
    pixel_area = hp.nside2pixarea(nside_skymap, degrees=True)
    probabilities = np.zeros(len(ra))
    vertices = hp.ang2vec([ra-0.5, ra-0.5, ra+0.5, ra+0.5], [dec-0.5, dec+0.5, dec+0.5, dec-0.5], lonlat=True)
    for i in range(len(ra)):
        #print(vertices[i,:])
        sel_pix = hp.query_polygon(nside_skymap, vertices[i,:], inclusive=True, fact=fact)
        probabilities[i] = np.sum(skymap[sel_pix]) * \
            tile_area / pixel_area / len(sel_pix)
    return probabilities



def get_top_tiles(probabilities, frac=0.99):
    """
    probabilities may not add up to 1
    return indices of tiles that add up to frac of total
    """
    sortorder = np.argsort(probabilities)
    p_cum = np.cumsum(probabilities[sortorder]) / np.sum(probabilities)
    startind = np.where(p_cum > 1 - frac)[0][0]
    top_tiles = sortorder[startind:]
    return np.flip(top_tiles)




In [ ]:
def get_top_tiles(probabilities, frac=0.99):
    """
    probabilities may not add up to 1
    return indices of tiles that add up to frac of total
    """
    sortorder = np.argsort(probabilities)
    p_cum = np.cumsum(probabilities[sortorder]) / np.sum(probabilities)
    startind = np.where(p_cum > 1 - frac)[0][0]
    top_tiles = sortorder[startind:]
    return np.flip(top_tiles)



dead2 = [2,17,23,27,28,33,35,39,44,47,49,50,54,55,79,80,86,88,89,90,94,99,
        110,111,112,129,130,142,152,154,162,165,167,168,174,179,180,189,191,
        213,215,219,224,227,231,232,244,254,267,274,276,280,294,295,
        312,313,335,338,340,342,361,363,380,383,385,388,393,395,
        409,432,433,437,440,446,449,453,456,492,493,496,497,498,
        513,516,522,523,524,539,549,550,552,553,564,565,567,569,572,575,581,582,583,586,591,
        603,605,607,610,653,655,659,661,675,676,678,681,689,693,694,
        905,914,916,921,923,924,925,927,928,933,942,943,953,954,960,961,963,964,966,984,985,989,
        1103,1110,1146,1154,1160,1166,1169,1174,1175,1178,1185,1190,1195,
        1205,1207,1212,1220,1221,1224,1235,1236,1240,1243,1253,1257,1270,1272,1273,1282,1288,
        1305,1307,1313,1325,1327,1326,1329,1336,1337,1339,1340,1357,1361,1371,1375,1377,1384,1386,
        1414,1439,1450,1470,1471,1497]





####  15, 48, 87, 25

In [ ]:
cat2 = pd.read_csv('../../new_reduced_galcat.csv')
new = pd.read_csv('../../missing_mass.csv')
cat2['Dist'] = cat2['DistMpc']

In [ ]:
d1 = pd.read_csv('full.csv')

names = d1['num'][:]

for name in names:
    
    skym = ligo.skymap.io.fits.read_sky_map('farah/allsky/{}.fits'.format(name), distances=True)
    skymap = hp.ud_grade(skym[0][0], 512)
    
    #print('\n\n\n',name,'\n\n\n')

    tiles = pd.read_csv('tiles_GIT_7.csv')
    ralist, declist = np.array(tiles['RA_Center'])*u.deg, np.array(tiles['DEC_Center'])*u.deg
    probs = get_probabilities(skymap, ralist, declist)
    tt = get_top_tiles(probs, 0.99)
    print(name,len(tt))

In [ ]:
cat2 = pd.read_csv('../../new_reduced_galcat.csv')
new = pd.read_csv('../../missing_mass.csv')
cat2['Dist'] = cat2['DistMpc']

In [ ]:
d1 = pd.read_csv('full.csv')

names = d1['num'][0:]

for name in names:
    
    skym = ligo.skymap.io.fits.read_sky_map('farah/allsky/{}.fits'.format(name), distances=True)
    skymap = hp.ud_grade(skym[0][0], 512)
    
    print('\n\n\n##############################',name,'#################################\n\n\n')

    tiles = pd.read_csv('tiles_WINTER.csv')
    ralist, declist = np.array(tiles['RA_Center']), np.array(tiles['DEC_Center'])
    probs = get_probabilities(skymap, ralist, declist)
    tt = get_top_tiles(probs, 0.99)
    len(tt)

    distmu = hp.ud_grade(skym[0][1], 512)
    distsigma = hp.ud_grade(skym[0][2], 512)
    prob2d = hp.ud_grade(skym[0][0], 512)


    radius = 0.35*u.deg
    fact = 1
    nside_skymap = 512

    tile_area = np.pi * radius.to(u.deg).value ** 2
    pixel_area = hp.nside2pixarea(nside_skymap, degrees=True)

    sigma_m = np.zeros(len(tt))

    mpcd = np.arange(0,1001,20)


    
    sigma_m_empty_nodist = np.zeros(len(tt))
    sigma_m_only = np.zeros(len(tt))
    
    #if len(tt)

    for h in range(len(tt)):
        ra = ralist[tt[h]]
        dec = declist[tt[h]]

        #vecs = hp.ang2vec(ra.to(u.deg).value, dec.to(u.deg).value, lonlat=True)

        vertices = hp.ang2vec([ra-0.6/np.cos(dec*np.pi/180), ra-0.6/np.cos(dec*np.pi/180), ra+0.6/np.cos(dec*np.pi/180), ra+0.6/np.cos(dec*np.pi/180)], [dec-0.5, dec+0.5, dec+0.5, dec-0.5], lonlat=True)
        sel_pix = hp.query_polygon(nside_skymap, vertices, inclusive=True, fact=fact)
        
        m,s = np.median(distmu[sel_pix]), np.median(distsigma[sel_pix])
        
        currcat = cat2[np.isin(np.array(cat2.PIX_ID_512), sel_pix)][['Dist','Mstar']]
        currcat = currcat[(currcat.Dist>=m-3*s) & (currcat.Dist<=m+3*s)]
        
        summass = np.sum(currcat['Mstar'])
        masses3 = summass * np.sum(prob2d[sel_pix])
        sigma_m_only[h] = summass
        sigma_m_empty_nodist[h] = masses3
        
        prob2d[sel_pix] = 0.0
        print(h)
        
    df = pd.DataFrame({'Unfilled2D':sigma_m_empty_nodist,'MassOnly_Distcut':sigma_m_only})
    df.to_csv('Orderings-2D-WINTER/Map_{}.csv'.format(name))

In [ ]:
d1 = pd.read_csv('full.csv')

names = d1['num']

for name in names[125:]:
    skym = ligo.skymap.io.fits.read_sky_map('farah/allsky/{}.fits'.format(name), distances=True)
    skymap = hp.ud_grade(skym[0][0], 512)
    
    print('\n\n\n##############################',name,'#################################\n\n\n')

    tiles = pd.read_csv('tiles_WINTER.csv')
    ralist, declist = np.array(tiles['RA_Center']), np.array(tiles['DEC_Center'])
    probs = get_probabilities(skymap, ralist, declist)
    tt = get_top_tiles(probs, 0.99)
    print(len(tt))

    distmu = hp.ud_grade(skym[0][1], 512)
    distsigma = hp.ud_grade(skym[0][2], 512)
    prob2d = hp.ud_grade(skym[0][0], 512)


    radius = 0.35*u.deg
    fact = 1
    nside_skymap = 512

    tile_area = np.pi * radius.to(u.deg).value ** 2
    pixel_area = hp.nside2pixarea(nside_skymap, degrees=True)

    sigma_m = np.zeros(len(tt))

    mpcd = np.arange(0,1001,20)


    sigma_m_empty_dist = np.zeros(len(tt))
    sigma_m_empty_nodist = np.zeros(len(tt))
    sigma_m_nonempty = np.zeros(len(tt))
    sigma_m_noprob = np.zeros(len(tt))
    sigma_m_noprob_filled = np.zeros(len(tt))
    
    for h in range(len(tt)):
        ra = ralist[tt[h]]
        dec = declist[tt[h]]
        
        vertices = hp.ang2vec([ra-0.6/np.cos(dec*np.pi/180), ra-0.6/np.cos(dec*np.pi/180), ra+0.6/np.cos(dec*np.pi/180), ra+0.6/np.cos(dec*np.pi/180)], [dec-0.5, dec+0.5, dec+0.5, dec-0.5], lonlat=True)
        sel_pix = hp.query_polygon(nside_skymap, vertices, inclusive=True, fact=fact)
        
        masses1 = np.zeros(len(sel_pix))
        masses2 = np.zeros(len(sel_pix))
        masses3 = np.zeros(len(sel_pix))
        masses4 = np.zeros(len(sel_pix))
        masses5 = np.zeros(len(sel_pix))

        mpcd = np.arange(0,1001,20)

        for j in range(len(sel_pix)):
            i = sel_pix[j]
            currcat = (cat2[cat2.PIX_ID_512 == i][['Dist','Mstar']])

            currcat.reset_index(drop=True, inplace=True)

            indices = np.arange(0,50,1)
            filled = []

            distfact = 1/distsigma[i] * np.exp( -(mpcd-distmu[i])**2 / (2*distsigma[i]**2) )

            for k in range(len(currcat)):
                ind = np.argmin(np.abs(new['distcen'] - currcat['Dist'][k]))
                filled.append(ind)

            unfilled = np.delete(indices, filled)
            masses1[j] = prob2d[i]*(np.sum(currcat['Mstar']*distfact[filled]) + np.sum(new['masstofill'][unfilled]*distfact[unfilled] / (new['num_empty_pixels'][unfilled])))

            masses2[j] = prob2d[i]*np.sum(currcat['Mstar']*distfact[filled])

            masses3[j] = prob2d[i]*np.sum(currcat['Mstar'])
            
            masses4[j] = np.sum(currcat['Mstar'])
            
            masses5[j] = masses4[j] + np.sum(new['masstofill'][unfilled] / (new['num_empty_pixels'][unfilled]))

        prob2d[sel_pix] = 0.0

        sigma_m_empty_dist[h] = np.sum(masses2)
        sigma_m_empty_nodist[h] = np.sum(masses3)
        sigma_m_nonempty[h] = np.sum(masses1)
        sigma_m_noprob[h] = np.sum(masses4)
        sigma_m_noprob_filled[h] = np.sum(masses5)
        print(h)
        
    df = pd.DataFrame({'Filled3D':sigma_m_nonempty, 'Unfilled3D':sigma_m_empty_dist, 'Unfilled2D':sigma_m_empty_nodist, 'NoProb':sigma_m_noprob, 'NoProbFilled':sigma_m_noprob_filled})
    df.to_csv('Ordering-WINTER/Map_{}.csv'.format(name))

In [ ]:
latest = 0

d1 = pd.read_csv('full.csv')
ar = []
dis = []
a1, a2, a3, a4, a5, a6 = [],[],[],[], [],[]
ref = pd.read_csv('full2.csv')
nobs = 100

for n in d1['num'][latest:]:
    cs = pd.read_csv('Ordering-WINTER/Map_{}.csv'.format(n))
    sigma_m_nonempty, sigma_m_empty_nodist, sigma_m_empty_dist, sigma_m_noprob = np.array(cs['Filled3D']), np.array(cs['Unfilled2D']), np.array(cs['Unfilled3D']), np.array(cs['NoProb'])
    fs = pd.read_csv('Orderings-2D-WINTER/Map_{}.csv'.format(n))
    sigma_2dmp, monly = np.array(fs['Unfilled2D']), np.array(fs['MassOnly_Distcut'])
    #sigma_m_empty_nodist = np.array(pd.read_csv('Orderings-2D/Map_{}.csv'.format(n))['Unfilled2D'])
    
    order_dist = np.flip(np.argsort(sigma_m_empty_dist)[-nobs:])
    order_nodist = np.flip(np.argsort(sigma_m_empty_nodist)[-nobs:])
    order_all = np.flip(np.argsort(sigma_m_nonempty)[-nobs:])
    order_noprob = np.flip(np.argsort(sigma_m_noprob)[-nobs:])
    order_2dmp = np.flip(np.argsort(sigma_2dmp)[-nobs:])
    order_mdcut = np.flip(np.argsort(monly)[-nobs:])

    fullsum = np.sum(sigma_m_nonempty)
    a1.append(np.cumsum((sigma_m_nonempty)[order_nodist])[-1] / fullsum)
    a2.append(np.cumsum((sigma_m_nonempty)[order_dist])[-1] / fullsum)
    a3.append(np.cumsum((sigma_m_nonempty)[order_all])[-1] / fullsum)
    a4.append(np.cumsum((sigma_m_nonempty)[order_noprob])[-1] / fullsum)
    a5.append(np.cumsum((sigma_m_nonempty)[order_2dmp])[-1] / fullsum)
    a6.append(np.cumsum((sigma_m_nonempty)[order_mdcut])[-1] / fullsum)
    print(n)
    #print(k, a3[-1], a1[-1], a2[-1])
        
    ar.append(0)
    
    dis.append(0)
    
d1['DegArea'][latest:] = ref['DegArea']
d1['Mean_Dist'][latest:] = ref['Mean_Dist']
d1['P_3dcat'][latest:] = a2
d1['P_2dcat_dcut'][latest:] = a5
d1['P_2dcat'][latest:] = a1
d1['P_massfill'][latest:] = a3
d1['P_onlymass'][latest:] = a4
d1['P_massdcut'][latest:] = a6

In [ ]:
latest = 0

d2 = pd.read_csv('full2.csv')
ar = []
dis = []
a1, a2, a3, a4 = [],[],[],[]
numtile = []
names = []

nobs = 100

for nam in d2['num'][latest:]:
    n = int(nam)
    cs = pd.read_csv('Ordering-WINTER/Map_{}.csv'.format(n))
    sigma_m_nonempty, sigma_m_empty_nodist, sigma_m_empty_dist, sigma_m_noprob = np.array(cs['Filled3D']), np.array(cs['Unfilled2D']), np.array(cs['Unfilled3D']), np.array(cs['NoProb'])

    order_dist = np.flip(np.argsort(sigma_m_empty_dist)[-nobs:])
    order_nodist = np.flip(np.argsort(sigma_m_empty_nodist)[-nobs:])
    order_all = np.flip(np.argsort(sigma_m_nonempty)[-nobs:])
    order_noprob = np.flip(np.argsort(sigma_m_noprob)[-nobs:])


    fullsum = np.sum(sigma_m_nonempty)
    a1.append(np.cumsum((sigma_m_nonempty)[order_nodist])[-1] / fullsum)
    a2.append(np.cumsum((sigma_m_nonempty)[order_dist])[-1] / fullsum)
    a3.append(np.cumsum((sigma_m_nonempty)[order_all])[-1] / fullsum)
    a4.append(np.cumsum((sigma_m_nonempty)[order_noprob])[-1] / fullsum)

    #print(k, a3[-1], a1[-1], a2[-1])
        
    if n in [511,559]:
        ar.append(0)
        dis.append(0)
        continue
        
    skym = ligo.skymap.io.fits.read_sky_map('farah/allsky/{}.fits'.format(n), distances=True)
    print(n)
    skymap = hp.ud_grade(skym[0][0], 512)
    #probs = get_probabilities(skymap, ralist, declist)
    tt = get_top_tiles(skymap, 0.99)
    a = len(tt)*hp.nside2pixarea(512)*3282.8
    ar.append(a)
    
    tt = get_top_tiles(skym[0][0])
    li = len(tt)
    tt = tt[~np.isinf(skym[0][1][tt])]
    dist = (np.sum(skym[0][0][tt]*skym[0][1][tt])/np.sum(skym[0][0][tt]))
    dis.append(dist)
    
d1['DegArea'][latest:] = ar
d1['Mean_Dist'][latest:] = dis
d1['P_3dcat'][latest:] = a1
d1['P_2dcat'][latest:] = a2
d1['P_massfill'][latest:] = a3
d1['P_onlymass'][latest:] = a4

In [ ]:
ar = np.array(ar)
dis = np.array(dis)
a1 = np.array(a1)
a2 = np.array(a2)
a3 = np.array(a3)

In [ ]:
d1 = pd.DataFrame()

d1['BNS_num'] = names
d1['Area99'] = numtile
d1['DegArea'] = ar[np.where(ar!=0)]
d1['Mean_Dist'] = dis[np.where(ar!=0)]
d1['P_3dcat'] = a1[np.where(ar!=0)]
d1['P_2dcat'] = a2[np.where(ar!=0)]
d1['P_massfill'] = a3[np.where(ar!=0)]

In [ ]:
#d1 = pd.read_csv('BNS_full.csv')
p2d = []

for nam in d1['num'][latest:]:
    if np.isnan(nam):
        continue
    n = int(nam)
    file = pd.read_csv('Ordering-WINTER/Map_{}.csv'.format(n))
    #print(len(file))
    p2 = np.sum(file.Filled3D[:100]) / np.sum(file.Filled3D[:])
    p2d.append(p2)
p2d    
d1['P_2d'] = np.array(p2d)#[np.where(ar!=0)]
d1

In [ ]:
d1.to_csv('Sept/full-WINTER100.csv')

### Contribution from Catalog and Missing:

In [ ]:
nobs = 100

kl = [33, 40, 55, 77, 88, 111]
#sigma_m = sigma_m[:1008]

for k in [33]:
    cs = pd.read_csv('BNS_GalCat/BNS_{}.csv'.format(k))
    sigma_m_nonempty, sigma_m_empty_nodist, sigma_m_empty_dist = np.array(cs['Filled3D']), np.array(cs['Unfilled2D']), np.array(cs['Unfilled3D'])

    order_dist = np.flip(np.argsort(sigma_m_empty_dist)[-nobs:])
    order_nodist = np.flip(np.argsort(sigma_m_empty_nodist)[-nobs:])
    order_all = np.flip(np.argsort(sigma_m_nonempty)[-nobs:])


    fullsum = np.sum(sigma_m_nonempty)
    a1 = np.cumsum((sigma_m_nonempty-sigma_m_empty_dist)[order_nodist]) / fullsum
    a2 = np.cumsum((sigma_m_nonempty-sigma_m_empty_dist)[order_dist]) / fullsum
    a3 = np.cumsum((sigma_m_nonempty-sigma_m_empty_dist)[order_all]) / fullsum
    
    b1 = np.cumsum((sigma_m_empty_dist)[order_nodist]) / fullsum
    b2 = np.cumsum((sigma_m_empty_dist)[order_dist]) / fullsum
    b3 = np.cumsum((sigma_m_empty_dist)[order_all]) / fullsum


    plt.figure(figsize=(11,8))
    plt.plot(a1, color='green', label='sorted by 3D prob, catalog galaxies')
    plt.plot(a2, color='orange', label = 'sorted by 2D prob, catalog galaxies')
    plt.plot(a3, color='darkorchid', label = 'sorted by 3D prob, mass filled')
    plt.plot(b1, color='green', linestyle='--')
    plt.plot(b2, color='orange', linestyle='--')
    plt.plot(b3, color='darkorchid', linestyle='--')
    plt.legend()
    plt.xlabel('Observation Number')
    plt.ylabel('3D prob*galaxy mass (filled+catalog) covered')
    plt.title('BNS {}'.format(k))
    plt.text(0,0.12, 'Solid - Catalog Galaxies, Dotted - Mass filled')
    plt.show()

    print(a3[-1]/a1[-1])

## BNS 559 - Catalog obscured by galactic plane!!

In [ ]:
tiles = pd.read_csv('tiles_GIT_7.csv')
ralist, declist = np.array(tiles['RA_Center'])*u.deg, np.array(tiles['DEC_Center'])*u.deg

nside_skymap = 512
radius = 0.35*u.deg
fact = 1

kl = [33, 40, 55, 77, 88, 111]
#sigma_m = sigma_m[:1008]

for k in [559]:
    cs = pd.read_csv('BNS_GalCat/BNS_{}.csv'.format(k))
    sigma_m_nonempty, sigma_m_empty_nodist, sigma_m_empty_dist = np.array(cs['Filled3D']), np.array(cs['Unfilled2D']), np.array(cs['Unfilled3D'])

    skym = ligo.skymap.io.fits.read_sky_map('bns_astro/allsky/{}.fits'.format(k), distances=True)
    skymap = hp.ud_grade(skym[0][0], 512)
    probs = get_probabilities(skymap, ralist, declist)
    tt = get_top_tiles(probs)
    
    z1 = np.zeros(12*512**2)
    z2 = np.zeros(12*512**2)
    
    fullsum = np.sum(sigma_m_nonempty)
    
    for h in range(len(tt)):
        ra = ralist[tt[h]]
        dec = declist[tt[h]]
        vecs = hp.ang2vec(ra.to(u.deg).value, dec.to(u.deg).value, lonlat=True)
        sel_pix = hp.query_disc(nside_skymap, vecs, radius.to(u.rad).value, inclusive=True, fact=fact)
        z1[sel_pix] = (sigma_m_nonempty - sigma_m_empty_dist)[h]/fullsum
        z2[sel_pix] = (sigma_m_empty_dist)[h]/fullsum
    
    hp.mollview(skymap)
    plt.title('BNS-{}-Original'.format(k))
    plt.show()
    hp.mollview(z1)
    plt.title('BNS-{}-Filled Mass'.format(k))
    plt.show()
    hp.mollview(z2)
    plt.title('BNS-{}-Cat Mass'.format(k))
    plt.show()

## BNS 357, 79 - Catalog along same LOS as below

In [ ]:
tiles = pd.read_csv('tiles_GIT_7.csv')
ralist, declist = np.array(tiles['RA_Center'])*u.deg, np.array(tiles['DEC_Center'])*u.deg

nside_skymap = 512
radius = 0.35*u.deg
fact = 1

kl = [33, 40, 55, 77, 88, 111]
#sigma_m = sigma_m[:1008]

for k in [79,357]:
    cs = pd.read_csv('BNS_GalCat/BNS_{}.csv'.format(k))
    sigma_m_nonempty, sigma_m_empty_nodist, sigma_m_empty_dist = np.array(cs['Filled3D']), np.array(cs['Unfilled2D']), np.array(cs['Unfilled3D'])

    skym = ligo.skymap.io.fits.read_sky_map('bns_astro/allsky/{}.fits'.format(k), distances=True)
    skymap = hp.ud_grade(skym[0][0], 512)
    probs = get_probabilities(skymap, ralist, declist)
    tt = get_top_tiles(probs)
    
    z1 = np.zeros(12*512**2)
    z2 = np.zeros(12*512**2)
    
    fullsum = np.sum(sigma_m_nonempty)
    
    for h in range(len(tt)):
        ra = ralist[tt[h]]
        dec = declist[tt[h]]
        vecs = hp.ang2vec(ra.to(u.deg).value, dec.to(u.deg).value, lonlat=True)
        sel_pix = hp.query_disc(nside_skymap, vecs, radius.to(u.rad).value, inclusive=True, fact=fact)
        z1[sel_pix] = (sigma_m_nonempty - sigma_m_empty_dist)[h]/fullsum
        z2[sel_pix] = (sigma_m_empty_dist)[h]/fullsum
    
    hp.mollview(skymap)
    plt.title('BNS-{}-Original'.format(k))
    plt.show()
    hp.mollview(z1)
    plt.title('BNS-{}-Filled Mass'.format(k))
    plt.show()
    hp.mollview(z2)
    plt.title('BNS-{}-Cat Mass'.format(k))
    plt.show()

## NSBH 931, 1388 - At same location, also galactic plane?

In [ ]:
tiles = pd.read_csv('tiles_GIT_7.csv')
ralist, declist = np.array(tiles['RA_Center'])*u.deg, np.array(tiles['DEC_Center'])*u.deg

nside_skymap = 512
radius = 0.35*u.deg
fact = 1

kl = [33, 40, 55, 77, 88, 111]
#sigma_m = sigma_m[:1008]

for k in [931, 1388, 1368]:
    cs = pd.read_csv('BNS_GalCat/NSBH_{}.csv'.format(k))
    sigma_m_nonempty, sigma_m_empty_nodist, sigma_m_empty_dist = np.array(cs['Filled3D']), np.array(cs['Unfilled2D']), np.array(cs['Unfilled3D'])

    skym = ligo.skymap.io.fits.read_sky_map('nsbh_astro/allsky/{}.fits'.format(k), distances=True)
    skymap = hp.ud_grade(skym[0][0], 512)
    probs = get_probabilities(skymap, ralist, declist)
    tt = get_top_tiles(probs)
    
    z1 = np.zeros(12*512**2)
    z2 = np.zeros(12*512**2)
    
    fullsum = np.sum(sigma_m_nonempty)
    
    for h in range(len(tt)):
        ra = ralist[tt[h]]
        dec = declist[tt[h]]
        vecs = hp.ang2vec(ra.to(u.deg).value, dec.to(u.deg).value, lonlat=True)
        sel_pix = hp.query_disc(nside_skymap, vecs, radius.to(u.rad).value, inclusive=True, fact=fact)
        z1[sel_pix] = (sigma_m_nonempty - sigma_m_empty_dist)[h]/fullsum
        z2[sel_pix] = (sigma_m_empty_dist)[h]/fullsum
    
    hp.mollview(skymap)
    plt.title('NSBH-{}-Original'.format(k))
    plt.show()
    hp.mollview(z1)
    plt.title('NSBH-{}-Filled Mass'.format(k))
    plt.show()
    hp.mollview(z2)
    plt.title('NSBH-{}-Cat Mass'.format(k))
    plt.show()

### Plots of Coverage by various sorting modes

In [ ]:
nobs = 100

kb = [33, 40, 55, 77, 88, 111, 81, 160, 238, 8, 334, 289]
kn = [120, 77, 300, 56, 69, 81, 633, 644, 447, 926, 74]
#sigma_m = sigma_m[:1008]

kb = [237,560,550,500,727,737,765]
kn = [177,971,273,730,876]

kn = [631,531,660,797,889,859,849,838]
kb = [347,358,415,448,977]

kn = [666, 632, 500,387]
kb = [1,487]
kn = [1005,1013,1014,1016,1021]

kn = [1090, 1055, 1040]

kb = list([1004,
360,
734,
847,
149,
225,
275,
425,
925,
930,
1045,
1088
])


kb = [1064,
1069,
1071,
1075,
1077,
1079,
1084,
1085,
1088,
1089
]

kn = [1096,
1092,
1085,
1082,
1067,
1062
]

kb = [707,
714,
717,
718,
721,
735,
748,
757
]

kn = [562,
565,
577,
582,
584,
586,
588,
594
]


for k in kn:
    cs = pd.read_csv('BNS_GalCat/BNS_{}.csv'.format(k))
    sigma_m_nonempty, sigma_m_empty_nodist, sigma_m_empty_dist = np.array(cs['Filled3D']), np.array(cs['Unfilled2D']), np.array(cs['Unfilled3D'])

    order_dist = np.flip(np.argsort(sigma_m_empty_dist)[-nobs:])
    order_nodist = np.flip(np.argsort(sigma_m_empty_nodist)[-nobs:])
    order_all = np.flip(np.argsort(sigma_m_nonempty)[-nobs:])


    fullsum = np.sum(sigma_m_nonempty)
    a1 = np.cumsum((sigma_m_nonempty)[order_nodist]) / fullsum
    a2 = np.cumsum((sigma_m_nonempty)[order_dist]) / fullsum
    a3 = np.cumsum((sigma_m_nonempty)[order_all]) / fullsum


    plt.figure(figsize=(11,8))
    plt.plot(a1, color='green', label='sorted by 3D prob, catalog galaxies')
    plt.plot(a2, color='orange', label = 'sorted by 2D prob, catalog galaxies')
    plt.plot(a3, color='darkorchid', label = 'sorted by 3D prob, mass filled')
    plt.legend()
    plt.xlabel('Observation Number')
    plt.ylabel('3D prob*galaxy mass (filled+catalog) covered')
    #plt.title('BNS {}'.format(k))
    #plt.show()

    print(k, a3[-1], a1[-1], a2[-1])

In [ ]:
latest = 234

d1 = pd.read_csv('NSBH_full.csv')
ar = []
dis = []
a1, a2, a3 = [],[],[]

nobs = 100

for n in d1['NSBH_num'][latest:]:
    cs = pd.read_csv('BNS_GalCat/NSBH_{}.csv'.format(n))
    sigma_m_nonempty, sigma_m_empty_nodist, sigma_m_empty_dist = np.array(cs['Filled3D']), np.array(cs['Unfilled2D']), np.array(cs['Unfilled3D'])

    order_dist = np.flip(np.argsort(sigma_m_empty_dist)[-nobs:])
    order_nodist = np.flip(np.argsort(sigma_m_empty_nodist)[-nobs:])
    order_all = np.flip(np.argsort(sigma_m_nonempty)[-nobs:])


    fullsum = np.sum(sigma_m_nonempty)
    a1.append(np.cumsum((sigma_m_nonempty)[order_nodist])[-1] / fullsum)
    a2.append(np.cumsum((sigma_m_nonempty)[order_dist])[-1] / fullsum)
    a3.append(np.cumsum((sigma_m_nonempty)[order_all])[-1] / fullsum)

    #print(k, a3[-1], a1[-1], a2[-1])
        
    if n in [511,559]:
        ar.append(0)
        dis.append(0)
        continue
        
    skym = ligo.skymap.io.fits.read_sky_map('nsbh_astro/allsky/{}.fits'.format(n), distances=True)
    print(n)
    skymap = hp.ud_grade(skym[0][0], 512)
    #probs = get_probabilities(skymap, ralist, declist)
    tt = get_top_tiles(skymap, 0.99)
    a = len(tt)*hp.nside2pixarea(512)*3282.8
    ar.append(a)
    
    tt = get_top_tiles(skym[0][0])
    li = len(tt)
    tt = tt[~np.isinf(skym[0][1][tt])]
    dist = (np.sum(skym[0][0][tt]*skym[0][1][tt])/np.sum(skym[0][0][tt]))
    dis.append(dist)
    
d1['DegArea'][latest:] = ar
d1['Mean_Dist'][latest:] = dis
d1['P_3dcat'][latest:] = a1
d1['P_2dcat'][latest:] = a2
d1['P_massfill'][latest:] = a3

In [ ]:
len(d1['P_massfill'][69:])

In [ ]:
a1

In [ ]:
#d1 = pd.read_csv('BNS_full.csv')
p2d = []

for n in d1['NSBH_num'][latest:]:
    file = pd.read_csv('BNS_GalCat/NSBH_{}.csv'.format(n))
    #print(len(file))
    p2 = np.sum(file.Filled3D[:100]) / np.sum(file.Filled3D[:])
    p2d.append(p2)
p2d    
d1['P_2d'][latest:] = p2d
d1

In [ ]:
d1= d1.sort_values('NSBH_num')
d1

In [ ]:
d1.to_csv('NSBH_full.csv')

In [ ]:
np.where(d1.P_massfill < d1.P_2d)

In [ ]:
f = pd.read_csv('')

In [ ]:
d1.to_csv('BNS_full.csv')

In [ ]:
n=512
12*n**2*hp.nside2pixarea(n)#, arcmin=True)

### Next Tasks:

1. Find a way to show why missing mass makes such a diference - 
a. plot skymaps with probabilities from all 3 components
b. show breakup of known / missing mass as dashed lines in cumu plot

2. look for trends as a function of median distance, 90% area - find out where 3d+missing makes most difference, where it is similar to 3d-g, 2d etc etc
a. 100-tile probab as a function of median distance / area
b. histograms of 3 probabs in different distance / area bins

### Overall Results